In [ ]:
# 설치 후 런타임 재실행 
!pip install -U segmentation-models-pytorch albumentations --user
!pip install --user albumentations
!pip install opencv-contrib-python

In [ ]:
import albumentations 
from albumentations.pytorch import ToTensorV2
from typing import List
import torch
import numpy as np
import torch.nn as nn
import torchvision
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as T
import segmentation_models_pytorch as smp
import os
import torchvision.transforms.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from matplotlib import colors, pyplot as plt
%matplotlib inline
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

## 1. 모델 정의 및 학습 
- 구글 마운트 후 사용하기

In [ ]:
# segmentation model - pytorch에서 제공
model = smp.Unet(
    encoder_name='resnext50_32x4d',
    classes=1,
    activation='sigmoid',
    encoder_weights='imagenet'
)

손 - 이미지, 마스크

In [ ]:
# 직접 찍은 손 사진 
dataset_dir = Path('/content/drive/MyDrive/4조/2.데이터')
hand_images_dir = dataset_dir / '손이미지'  # 직접 찍은 손 사진 
hand_masks_dir = dataset_dir / '손톱마스크' # 직접 만든 손 사진 마스크

hand_images_lst = [filename for filename in os.listdir(hand_images_dir)]
hand_masks_lst = [filename for filename in os.listdir(hand_masks_dir)]

len(hand_images_lst), len(hand_masks_lst)

In [ ]:
# 마스크 이미지와 중복되는 데이터만 사용 
hand_images_n = [img for img in hand_images_lst if img in hand_masks_lst]
len(hand_images_n)

In [ ]:
# images, masks = [], []
images = sorted([hand_images_dir / filename for filename in hand_images_n])
masks = sorted([hand_masks_dir / filename for filename in hand_images_n])
assert len(images) == len(masks)

추가 손 데이터

In [ ]:
# 추가 데이터 
dataset_dir = Path('/content/drive/MyDrive/프포젝트/3_DDD/archive/nails_segmentation')
images_dir = dataset_dir / 'images'
masks_dir = dataset_dir / 'labels'

images_add = sorted([images_dir / filename for filename in os.listdir(images_dir)])
masks_add  = sorted([masks_dir / filename for filename in os.listdir(masks_dir)])

images.extend(images_add) 
masks.extend(masks_add)

발 - 이미지, 마스크

In [ ]:
# 직접 찍은 발 사진 
dataset_dir = Path('/content/drive/MyDrive/4조/2.데이터')   # 발톱 사진
# my_dir = Path('/content/drive/MyDrive/프포젝트/3_DDD/data') # 밡톱마스크는 내 드라이브에
foot_images_dir = dataset_dir / '발이미지'  # 직접 찍은 발 사진 
foot_masks_dir = dataset_dir / '발톱마스크' # 직접 찍은 발 사진 마스크

foot_images_lst = [filename for filename in os.listdir(foot_images_dir)]
foot_masks_lst = [filename for filename in os.listdir(foot_masks_dir)]

len(foot_images_lst), len(foot_masks_lst)

In [ ]:
# 마스크 이미지와 중복되는 데이터만 사용 
foot_images_n = [img for img in foot_images_lst if img in foot_masks_lst]
len(foot_images_n)

In [ ]:
foot_images_add = sorted([foot_images_dir / filename for filename in foot_images_n])
foot_masks_add  = sorted([foot_masks_dir / filename for filename in foot_images_n])
images.extend(foot_images_add) 
masks.extend(foot_masks_add)

In [ ]:
len(images), len(masks)

train, test, val data split

In [ ]:
# train, test, val data split
images_train, images_valid, masks_train, masks_valid = train_test_split(images, masks, train_size=0.7, shuffle=False)

In [ ]:
class NailsDataset(torch.utils.data.Dataset):
    def __init__(self, images: List[str], masks: List[str], transform):
        self._images = images
        self._masks = masks
        self._transform = transform
        assert len(images) == len(masks)
  
    def __len__(self):
        return len(self._images)
  
    def __getitem__(self, index):
        image, mask = Image.open(self._images[index]), Image.open(self._masks[index]).convert('L')
        augmented = self._transform(image=np.array(image), mask=np.array(mask))
        image, mask = augmented['image'], augmented['mask']
        return image, mask[None].float() / 255

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
loss = smp.utils.losses.DiceLoss() + smp.utils.losses.BCELoss()
metrics = [smp.utils.metrics.IoU(threshold=0.47)]

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.005)
#lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2, eta_min=5e-5)

In [ ]:
trainepoch = smp.utils.train.TrainEpoch(
    model,
    loss=loss,
    optimizer=optimizer,
    metrics=metrics,
    device=device,
    verbose=True,
)

validepoch = smp.utils.train.ValidEpoch(
    model,
    loss=loss,
    metrics=metrics,
    device=device,
    verbose=True,
)

In [ ]:
epochs = 35
train_logs_list, valid_logs_list = [], []

In [ ]:
max_score = 0
for epoch in range(epochs):
    print(f'Epoch: {epoch}')
    
    dataset_train = NailsDataset(
    images=images_train,
    masks=masks_train,
    transform=A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.RandomResizedCrop(height=224, width=224, scale=(0.7, 1)),
        A.Resize(224, 224),
        A.Flip(p=1),
        A.Rotate(limit=15, p=1),
        A.Sharpen(p=0.5),
        ToTensorV2(transpose_mask=True),
    ]),
    )

    dataset_valid = NailsDataset(
        images=images_valid,
        masks=masks_valid,
        transform=A.Compose([
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            A.Resize(224, 224),
            ToTensorV2(transpose_mask=True),
        ]),
    )
    dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=4, shuffle=True)
    dataloader_valid = torch.utils.data.DataLoader(dataset_valid, batch_size=4, shuffle=True)
    
    trainlogs = trainepoch.run(dataloader_train)
    validlogs = validepoch.run(dataloader_valid)

    train_logs_list.append(trainlogs)
    valid_logs_list.append(validlogs)
    
    if max_score < validlxfogs['iou_score']:
        max_score = validlogs['iou_score']
        torch.save(model, '/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model_2.pth')
        print('Model saved!')

저장된 모델 불러와 결과 확인 


모델2

In [ ]:
# 저장된 모델 불러오기
best_model = torch.load('/content/drive/MyDrive/4조/2.데이터/마스크 생성 모델/nail_foot_seg_best_model_2.pth') # pytorch model 확장자 .pth

In [ ]:
def iou(target, prediction):
    prediction = prediction
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def iou2(target, prediction):
    tp = torch.sum(preds * y)  # TP
    fp = torch.sum(preds * (1 - y))  # FP
    fn = torch.sum((1 - preds) * y)  # FN
    tn = torch.sum((1 - preds) * (1 - y))  # TN

    iou_score2 = (tp + 1e-8) / (tp + fp + fn + 1e-8)
    mean_acc = (tp + 1e-8)/(tp+fn + 1e-8)
    
    return iou_score2, mean_acc

In [ ]:
from skimage.filters import threshold_otsu
from torchvision.utils import draw_segmentation_masks
iou_score = 0
iou_score2 = 0
mean_acc = 0
otsu = 0
for imgs, masks in dataloader_valid:
    for img, mask in zip(imgs, masks):
        img = torch.unsqueeze(img, 0)  # torch.unsqueeze(): 1차원 제거 함수
        mask = torch.unsqueeze(mask, 0)
        
        fig, axes = plt.subplots(1, 3)
        pred = best_model((img).to(device))[0][0].cpu().detach().numpy() > 0.47
        preds = (best_model((img).to(device))[0][0].cpu().detach() > 0.47).float()
        y = (((mask).to(device))[0][0].cpu().detach()).float()

        axes[0].imshow(pred)
        axes[0].set_title("Predicted mask")
        axes[1].imshow(((mask).to(device))[0][0].cpu().detach().numpy())
        axes[1].set_title("Target mask")
        axes[2].imshow(((img).to(device))[0][0].cpu().detach().numpy())
        axes[2].set_title("Target image")

        iou_score += iou(((mask).to(device))[0][0].cpu().detach().numpy(),pred)
        print("IoU", iou(((mask).to(device))[0][0].cpu().detach().numpy(),pred))

        otsu += threshold_otsu(best_model((img).to(device))[0][0].cpu().detach().numpy())
        iou_2, acc = iou2(((mask).to(device))[0][0].cpu().detach().numpy(),pred)
        iou_score2 += iou_2
        mean_acc += acc

        fig.set_figwidth(12)   
        fig.set_figheight(6)  
        plt.show()

iou_score = iou_score/len(dataset_valid)
iou_score2 = iou_score2/len(dataset_valid)
mean_acc = mean_acc/len(dataset_valid)
otsu = otsu/16
print("평균 IoU", iou_score)
print("평균 IoU2", iou_score2.item())
print("mean accuracy", mean_acc.item())
print(otsu)

모델1


In [ ]:
best_model = torch.load('/content/drive/MyDrive/프포젝트/3_DDD/data/nail_foot_seg_best_model.pth') # pytorch model 확장자 .pth

from skimage.filters import threshold_otsu
from torchvision.utils import draw_segmentation_masks
iou_score = 0
iou_score2 = 0
mean_acc = 0
otsu = 0
for imgs, masks in dataloader_valid:
    for img, mask in zip(imgs, masks):
        img = torch.unsqueeze(img, 0)  # torch.unsqueeze(): 1차원 제거 함수
        mask = torch.unsqueeze(mask, 0)
        
        fig, axes = plt.subplots(1, 3)
        pred = best_model((img).to(device))[0][0].cpu().detach().numpy() > 0.47
        preds = (best_model((img).to(device))[0][0].cpu().detach() > 0.47).float()
        y = (((mask).to(device))[0][0].cpu().detach()).float()

        axes[0].imshow(pred)
        axes[0].set_title("Predicted mask")
        axes[1].imshow(((mask).to(device))[0][0].cpu().detach().numpy())
        axes[1].set_title("Target mask")
        axes[2].imshow(((img).to(device))[0][0].cpu().detach().numpy())
        axes[2].set_title("Target image")

        iou_score += iou(((mask).to(device))[0][0].cpu().detach().numpy(),pred)
        print("IoU", iou(((mask).to(device))[0][0].cpu().detach().numpy(),pred))

        otsu += threshold_otsu(best_model((img).to(device))[0][0].cpu().detach().numpy())
        iou_2, acc = iou2(((mask).to(device))[0][0].cpu().detach().numpy(),pred)
        iou_score2 += iou_2
        mean_acc += acc

        fig.set_figwidth(12)   
        fig.set_figheight(6)  
        plt.show()

iou_score = iou_score/len(dataset_valid)
iou_score2 = iou_score2/len(dataset_valid)
mean_acc = mean_acc/len(dataset_valid)
otsu = otsu/16
print("평균 IoU", iou_score)
print("평균 IoU2", iou_score2.item())
print("mean accuracy", mean_acc.item())
print(otsu)

## 2. 마스크 없는 사진으로 테스트
- 저장된 모델 불러와서 사용하기

In [ ]:
# 저장된 모델 불러오기
# 사용시 경로 변경 후 사용!
best_model = torch.load('/content/drive/MyDrive/프포젝트/3_DDD/data/nail_foot_seg_best_model.pth') # pytorch model 확장자 .pth

In [ ]:
# 데이터 불러오기 
test_dataset_dir = Path('/content/drive/MyDrive/4조/2.데이터')
test_hand_images_dir = test_dataset_dir / '손이미지'
test_foot_images_dir = test_dataset_dir / '발이미지'

test_hand_images = sorted([test_hand_images_dir / filename for filename in os.listdir(test_hand_images_dir)])
test_foot_images = sorted([test_foot_images_dir / filename for filename in os.listdir(test_foot_images_dir)])

In [ ]:
import cv2
img = cv2.imread("/content/drive/MyDrive/4조/2.데이터/손이미지/손_0.jpg")
img.shape

In [ ]:
len(test_hand_images), len(test_foot_images)

In [ ]:
test_hand_images[0]

In [ ]:
class test_NailsDataset(torch.utils.data.Dataset):
    def __init__(self, images: List[str], transform):
        self._images = images
        # self._masks = masks
        self._transform = transform
        # assert len(images) == len(masks)
  
    def __len__(self):
        return len(self._images)
  
    def __getitem__(self, index):
        image = Image.open(self._images[index])
        augmented = self._transform(image=np.array(image))
        image= augmented['image']
        return image

손 이미지

In [ ]:
test_dataset = test_NailsDataset(
    images=test_hand_images,
    transform=A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.Resize(height= 224 ,width= 224),
        # A.RandomResizedCrop(height=224, width=224, scale=(0.7, 1)),
        # A.Flip(p=1),
        # A.Rotate(limit=15, p=1),
        A.Sharpen(p=0.5),
        ToTensorV2(transpose_mask=True),
    ]),
    )

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1 ,shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
len(test_dataset) ,len(test_dataloader)

In [ ]:
# def inverse_normalize(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
#     """
#     :param img: numpy array. shape (height, width, channel). [-1~1]
#     :return: numpy array. shape (height, width, channel). [0~1]
#     """
#     img[:,:,0] = ((img[:,:,0]) * std[0]) + mean[0]
#     img[:,:,1] = ((img[:,:,1]) * std[1]) + mean[1]
#     img[:,:,2] = ((img[:,:,2]) * std[2]) + mean[2]
#     return img

In [ ]:
### TEST DATA ###
# score 표시 X
from skimage.filters import threshold_otsu
from torchvision.utils import draw_segmentation_masks
from tqdm import tqdm
from PIL import Image

iou_score = 0
iou_score2 = 0
mean_acc = 0
otsu = 0

for i, imgs in tqdm(enumerate(test_dataloader)):
    for img in imgs:
      
        img = torch.unsqueeze(img, 0)  # torch.unsqueeze(): 1차원 제거 함수
        pred = best_model((img).to(device))[0][0].cpu().detach().numpy() > 0.47  
        preds = (best_model((img).to(device))[0][0].cpu().detach() > 0.47).float()
        # y = (((mask).to(device))[0][0].cpu().detach()).float()

        # 시각화 
        fig, axes = plt.subplots(1, 2)
        axes[0].imshow(pred)
        axes[0].set_title("Predicted mask")

        # axes[1].imshow(((img).to(device))[0][0].cpu().detach().numpy())
        img_o = ((img).to(device))[0][0].cpu().detach().numpy()
        axes[1].imshow(img_o)
        axes[1].set_title("Target image")

        otsu += threshold_otsu(best_model((img).to(device))[0][0].cpu().detach().numpy())
        fig.set_figwidth(12)   
        fig.set_figheight(6)  
        plt.show()
        
        # img_og = cv2.cvtColor(np.array(img_o), cv2.COLOR_BGR2RGB)
        # PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seed:04d}.png')
        # image = Image.fromarray(np.uint8(cm.plasma(image_array)*255))
        # Image.fromarray(np.uint8(cm.plasma(img_o)*255)).save(f'/content/drive/MyDrive/4조/2.데이터/모델_손발마스크/손/손_원본/원본_손_{i}.jpg')
        
        # plt.imsave( f'/content/drive/MyDrive/4조/2.데이터/모델_손발마스크/손/손_원본/원본_손_{i}.jpg', img_og)
        plt.imsave( f'/content/drive/MyDrive/4조/2.데이터/모델_손발마스크/손/모델마스크_손_{i}.jpg', pred, cmap='gray')



발 이미지

In [ ]:
test_dataset = test_NailsDataset(
    images=test_foot_images,
    transform=A.Compose([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.Resize(height= 224 ,width= 224),
        A.Sharpen(p=0.5),
        ToTensorV2(transpose_mask=True),
    ]),
    )

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
### TEST DATA ###
# score 표시 X
from skimage.filters import threshold_otsu
from torchvision.utils import draw_segmentation_masks
from tqdm import tqdm

iou_score = 0
iou_score2 = 0
mean_acc = 0
otsu = 0

test_imgs_lst, test_preds_lst = [], [] # 원본 이미지, 마스크 이미지 담을 리스트 

for i, imgs in tqdm(enumerate(test_dataloader)):
    for img in imgs:
      
        img = torch.unsqueeze(img, 0)  # torch.unsqueeze(): 1차원 제거 함수
        pred = best_model((img).to(device))[0][0].cpu().detach().numpy() > 0.47  
        preds = (best_model((img).to(device))[0][0].cpu().detach() > 0.47).float()
        # y = (((mask).to(device))[0][0].cpu().detach()).float()

        test_imgs_lst.append(img)
        test_preds_lst.append(preds)

        # 시각화 
        fig, axes = plt.subplots(1, 2)
        axes[0].imshow(pred)
        axes[0].set_title("Predicted mask")
        axes[1].imshow(((img).to(device))[0][0].cpu().detach().numpy())
        axes[1].set_title("Target image")

        otsu += threshold_otsu(best_model((img).to(device))[0][0].cpu().detach().numpy())
        fig.set_figwidth(12)   
        fig.set_figheight(6)  
        plt.show()
        
        # 사진 저장
        plt.imsave( f'/content/drive/MyDrive/4조/2.데이터/모델_손발마스크/발/모델마스크_발_{i}.jpg', pred, cmap='gray')